<a href="https://colab.research.google.com/github/Lily-2002/Retrosynthetic_Planning/blob/main/test_mT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit
!pip install --no-cache-dir transformers sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [ ]:
from rdkit import Chem
def get_examples(n_examples,path):
    import numpy as np
    import random
    data  = pd.read_csv(path)
    examples = "Please predict the reactants of products.\n"
    unique_elements, indices = np.unique(np.array(list(data["retro_templates"])), return_index=True)
    # print(indices)
    indices = random.sample(list(indices),n_examples)
    # print(indices)
    for indice in indices:
        # print(indice)
        examples += "products:" + data.loc[indice,"products"] + " " + "reactants:" + data.loc[indice,"reactants"] + "\n"
# print(examples)
    return examples,indices

def extract_reactants_after_pattern(main_string,standard):

    print(f"main_string: {main_string}")
    pattern_string = "<extra_id_0>"
    index = main_string.find(pattern_string)
    if main_string.find("reactants:") != -1:
      reatants = main_string[index + 12: main_string.find("reactants:")]
    else:
      reatants = main_string[index + 12 :]
    reatants = reatants.replace("\n","")
    reatants = reatants.replace(" ","")
    print("reatants")
    print(reatants)
    # print("standard")
    # print(standard)
    if is_valid_smiles(reatants) == False:
      return 0
    else:
      mol = Chem.MolFromSmiles(reatants)
      # print(mol)
      mol_stan = Chem.MolFromSmiles(standard)
      # print(mol_stan)
      if Chem.MolToInchi(mol) == Chem.MolToInchi(mol_stan):
          return 1
      else:
          return 0
def eval(path,example,used_indices):
    data = pd.read_csv(path)
    acc = 0
    for index,row in data.iterrows():
        if index not in used_indices:
            prompt = example +"products:" + row["products"] + " " + "reactants:"
            # print(prompt)
            prompt = prompt +" <extra_id_0>"
            inputs = tokenizer(prompt, return_tensors='pt')
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            outputs =  model.generate(
                            input_ids,
                            attention_mask=attention_mask,
                            top_p=0.9,
                            temperature = 0.1,
                            num_beams=8,
                            max_new_tokens=256,
                            num_return_sequences=8
                    )
            sum = 0
            for i, output in enumerate(outputs):
                generated_text = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
                print(generated_text)
                a = extract_reactants_after_pattern(main_string = generated_text,standard=row["reactants"])
                sum = sum + a
            sum = sum / 8
            acc = acc + sum
            print(f"ACC: {acc}")
    # print(acc)
    return
if __name__ == '__main__':
    import pandas as pd
    from transformers import AutoTokenizer,MT5ForConditionalGeneration
    tokenizer = AutoTokenizer.from_pretrained("/content/gdrive/MyDrive/mT5-small",use_fast=False)
    model = MT5ForConditionalGeneration.from_pretrained("/content/gdrive/MyDrive/mT5-small")
    exp,idx  = get_examples(n_examples=4,path="/content/gdrive/MyDrive/syndata_for_enzyme.csv")
    eval(path="/content/gdrive/MyDrive/syndata_for_enzyme.csv",example=exp,used_indices=idx)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<extra_id_0> C[C@H](C)OC=O
main_string: <extra_id_0> C[C@H](C)OC=O
reatants
C[C@H](C)OC=O
<extra_id_0> O=P1(O)O
main_string: <extra_id_0> O=P1(O)O
reatants
O=P1(O)O
<extra_id_0> C(=O)O
main_string: <extra_id_0> C(=O)O
reatants
C(=O)O
<extra_id_0> O=C(O)O
main_string: <extra_id_0> O=C(O)O
reatants
O=C(O)O
<extra_id_0> O=P1(O)OC=O
main_string: <extra_id_0> O=P1(O)OC=O
reatants
O=P1(O)OC=O
<extra_id_0> NC(=O)O
main_string: <extra_id_0> NC(=O)O
reatants
NC(=O)O
<extra_id_0> NC(=O)OC=O
main_string: <extra_id_0> NC(=O)OC=O
reatants
NC(=O)OC=O
<extra_id_0> C(O)OC=O
main_string: <extra_id_0> C(O)OC=O
reatants
C(O)OC=O
ACC: 0.0


[12:34:46] SMILES Parse Error: unclosed ring for input: 'O=P1(O)O'
[12:34:46] SMILES Parse Error: unclosed ring for input: 'O=P1(O)OC=O'


<extra_id_0> CC[C@H](C)OC=O
main_string: <extra_id_0> CC[C@H](C)OC=O
reatants
CC[C@H](C)OC=O
<extra_id_0> CC[C@@H](C)OC=O
main_string: <extra_id_0> CC[C@@H](C)OC=O
reatants
CC[C@@H](C)OC=O
<extra_id_0> CC[C@H](C)O1
main_string: <extra_id_0> CC[C@H](C)O1
reatants
CC[C@H](C)O1
<extra_id_0> CC[C@H](C)OC=O reactants:
main_string: <extra_id_0> CC[C@H](C)OC=O reactants:
reatants
CC[C@H](C)OC=O
<extra_id_0> O=P1(O)O
main_string: <extra_id_0> O=P1(O)O
reatants
O=P1(O)O
<extra_id_0> CC[C@H](C)O
main_string: <extra_id_0> CC[C@H](C)O
reatants
CC[C@H](C)O
<extra_id_0> O=C(O)O
main_string: <extra_id_0> O=C(O)O
reatants
O=C(O)O
<extra_id_0> CC[C@H](O)OC=O
main_string: <extra_id_0> CC[C@H](O)OC=O
reatants
CC[C@H](O)OC=O
ACC: 0.125


[12:34:50] SMILES Parse Error: unclosed ring for input: 'CC[C@H](C)O1'
[12:34:50] SMILES Parse Error: unclosed ring for input: 'O=P1(O)O'


...

[12:42:47] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(=O)O'
[12:42:47] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(=O)OC'
[12:42:47] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(O)O'
[12:42:47] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)'
[12:42:47] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@@H](N)C(=O)O'


<extra_id_0> NC(=O)OP(=O)O
main_string: <extra_id_0> NC(=O)OP(=O)O
reatants
NC(=O)OP(=O)O
<extra_id_0> NC(=O)OP(=O)(O)O
main_string: <extra_id_0> NC(=O)OP(=O)(O)O
reatants
NC(=O)OP(=O)(O)O
<extra_id_0> NC(=NCCC[C@H](N)C(=O)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)C(=O)O
reatants
NC(=NCCC[C@H](N)C(=O)O
<extra_id_0> NC(=NCCC[C@H](CC(=O)OC)C(=O)O
main_string: <extra_id_0> NC(=NCCC[C@H](CC(=O)OC)C(=O)O
reatants
NC(=NCCC[C@H](CC(=O)OC)C(=O)O
<extra_id_0> NC(=NCCC[C@H](N)C(O)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)C(O)O
reatants
NC(=NCCC[C@H](N)C(O)O
<extra_id_0> O=C(O)O
main_string: <extra_id_0> O=C(O)O
reatants
O=C(O)O
<extra_id_0> NC(=O)O
main_string: <extra_id_0> NC(=O)O
reatants
NC(=O)O
<extra_id_0> NC(=O)OP(=O)O products: O=C(O)O
main_string: <extra_id_0> NC(=O)OP(=O)O products: O=C(O)O
reatants
NC(=O)OP(=O)Oproducts:O=C(O)O
ACC: 2.25


[12:42:52] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(=O)O'
[12:42:52] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](CC(=O)OC)C(=O)O'
[12:42:52] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(O)O'
[12:42:52] SMILES Parse Error: syntax error while parsing: NC(=O)OP(=O)Oproducts:O=C(O)O
[12:42:52] SMILES Parse Error: Failed parsing SMILES 'NC(=O)OP(=O)Oproducts:O=C(O)O' for input: 'NC(=O)OP(=O)Oproducts:O=C(O)O'


<extra_id_0> NC(=O)OP(=O)O
main_string: <extra_id_0> NC(=O)OP(=O)O
reatants
NC(=O)OP(=O)O
<extra_id_0> NC(=NCCC[C@H](N)C(=O)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)C(=O)O
reatants
NC(=NCCC[C@H](N)C(=O)O
<extra_id_0> NC(=NCCC[C@H](N)C(O)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)C(O)O
reatants
NC(=NCCC[C@H](N)C(O)O
<extra_id_0> NC(=O)OP(=O)(O)O
main_string: <extra_id_0> NC(=O)OP(=O)(O)O
reatants
NC(=O)OP(=O)(O)O
<extra_id_0> NC(=NCCC[C@H](N)OC)C(=O)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)OC)C(=O)O
reatants
NC(=NCCC[C@H](N)OC)C(=O)O
<extra_id_0> NC(=NCCC[C@@H](N)C(=O)O
main_string: <extra_id_0> NC(=NCCC[C@@H](N)C(=O)O
reatants
NC(=NCCC[C@@H](N)C(=O)O
<extra_id_0> NC(=NCCC[C@H](N)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)O
reatants
NC(=NCCC[C@H](N)O
<extra_id_0> NC(=NCCC[C@H](N)OC)C(O)O
main_string: <extra_id_0> NC(=NCCC[C@H](N)OC)C(O)O
reatants
NC(=NCCC[C@H](N)OC)C(O)O
ACC: 2.25


[12:42:58] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(=O)O'
[12:42:58] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)C(O)O'
[12:42:58] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@@H](N)C(=O)O'
[12:42:58] SMILES Parse Error: extra open parentheses for input: 'NC(=NCCC[C@H](N)O'


In [ ]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/syndata_for_enzyme.csv")
result = 2.25 / len(list(data["reactants"]))
print(result)

0.021634615384615384
